In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Concatenate
from keras.optimizers import Adam
from keras.utils import to_categorical

In [ ]:
# Set the path to the directory containing the digit images
image_directory = "/content/drive/MyDrive/ANN/train"

In [ ]:
# Define the image size for resizing
image_size = (28, 28)

In [ ]:
# Initialize lists to store the images and labels
images = []
labels = []

In [ ]:
# Load and preprocess the images
for image_file in image_files:
    image_path = os.path.join(image_directory, image_file)
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    image = image.resize(image_size)
    image_array = np.array(image) / 255.0  # Normalize pixel values
    images.append(image_array)

    # Extract the label from the image file name
    label = int(image_file.split("_")[0])
    labels.append(label)

# Convert the images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Expand dimensions for CNN input
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)

# Encode the labels using one-hot encoding
y_train = to_categorical(y_train, num_classes=10)
y_val = to_categorical(y_val, num_classes=10)

In [ ]:
# Define the number of sub-networks (one for each digit)
num_subnetworks = 10

In [ ]:
# Define the input layer
input_layer = Input(shape=(image_size[0], image_size[1], 1))

# Define the list of sub-networks
subnetworks = []

In [ ]:
# Create the sub-networks
for i in range(num_subnetworks):
    # Convolutional layers
    conv1 = Conv2D(32, kernel_size=(3, 3), activation="relu")(input_layer)
    maxpool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, kernel_size=(3, 3), activation="relu")(maxpool1)
    maxpool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    flatten = Flatten()(maxpool2)

    # Fully connected layers
    dense1 = Dense(64, activation="relu")(flatten)
    output_layer = Dense(1, activation="sigmoid")(dense1)
    subnetworks.append(Model(inputs=input_layer, outputs=output_layer))

In [ ]:
# Concatenate the outputs of all sub-networks
concatenated_outputs = Concatenate()(subnetworks)

# Define the final prediction layer
prediction_layer = Dense(num_subnetworks, activation="softmax")(concatenated_outputs)

# Create the Group-connected MLP model
model = Model(inputs=input_layer, outputs=prediction_layer)

# Compile the model
model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=10)

# Evaluate the model on a separate test set if available

# Make predictions on new test images
# test_images = ... (preprocess the test images as done for training images)
# test_images = np.expand_dims(test_images, axis=-1)
# predictions = model.predict(test_images)
# predicted_digits = np.argmax(predictions, axis=1)
# print(predicted_digits)